In [14]:
# import getpass
# import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [15]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import Settings
import chromadb
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding



In [16]:
# Set prompt template for generation (optional)
from llama_index.core import PromptTemplate

def completion_to_prompt(completion):
   return f"<|im_start|>system\n<|im_end|>\n<|im_start|>user\n{completion}<|im_end|>\n<|im_start|>assistant\n"

def messages_to_prompt(messages):
    prompt = ""
    for message in messages:
        if message.role == "system":
            prompt += f"<|im_start|>system\n{message.content}<|im_end|>\n"
        elif message.role == "user":
            prompt += f"<|im_start|>user\n{message.content}<|im_end|>\n"
        elif message.role == "assistant":
            prompt += f"<|im_start|>assistant\n{message.content}<|im_end|>\n"

    if not prompt.startswith("<|im_start|>system"):
        prompt = "<|im_start|>system\n" + prompt

    prompt = prompt + "<|im_start|>assistant\n"

    return prompt

# Set Qwen2.5 as the language model and set generation config
Settings.llm = HuggingFaceLLM(
    model_name="Qwen/Qwen2.5-0.5B",
    tokenizer_name="Qwen/Qwen2.5-0.5B",
    context_window=30000,
    max_new_tokens=2000,
    generate_kwargs={"temperature": 0.7, "top_k": 50, "top_p": 0.95},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    device_map="auto",
)

# Set embedding model
Settings.embed_model = HuggingFaceEmbedding(
    model_name = "all-MiniLM-L6-v2"
)

# Set the size of the text chunk for retrieval
Settings.transformations = [SentenceSplitter(chunk_size=512)]

/home/drissdo/anaconda3/envs/thesis/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [17]:
documents = SimpleDirectoryReader("../data").load_data()
index = VectorStoreIndex.from_documents(
    documents,
)

In [18]:
text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=10)
Settings.text_splitter = text_splitter

# per-index
index = VectorStoreIndex.from_documents(
    documents, transformations=[text_splitter]
)

In [19]:
# initialize client, setting path to save data
db = chromadb.PersistentClient(path="./chroma_db")

# create collection
chroma_collection = db.get_or_create_collection("quickstart") 

In [20]:
# assign chroma as the vector_store to the context
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# create your index
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

NameError: name 'vector_store' is not defined

In [64]:
# create a query engine
query_engine = index.as_query_engine()
response = query_engine.query("Introduction about university?")
print(response)

The International University, established in December 2003, is Vietnam's only multidisciplinary public university that conducts teaching and research entirely in English. It offers a variety of undergraduate and postgraduate programs with a focus on economics, management, and engineering technology. The university aims to meet international standards and collaborates with prestigious institutions from the United States, Europe, and the Asia-Pacific region. Its goal is to become a leading research university in Vietnam and the region, providing high-quality human resources for integration. The university offers 23 training programs independently and 20 in partnership with esteemed partners, covering a wide range of majors such as Marketing, Statistics, Chemical Engineering, Data Science, and many more.


In [ ]:
query_engine = index.as_query_engine()
your_query = "<your query here>"
print(query_engine.query(your_query).response)